In [62]:
import os
import pandas as pd
from ast import literal_eval

In [63]:
combined = []

for dir_ in os.listdir("agreement_analysis"):
    d = pd.read_csv(f"agreement_analysis/{dir_}/all.csv")
    d["type"] = dir_
    combined.append(d)

combined = pd.concat(combined)

In [64]:
mask = combined.combination.apply(lambda x: 'gpt_' in x)

In [65]:
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

In [66]:
best_values_per_model = combined[combined.no_models==1].groupby('combination').max()[['mcc','f1','p','r']].reset_index()
best_values_per_model = best_values_per_model.sort_values(by='mcc',ascending=False)
best_values_per_model['combination']=best_values_per_model['combination'].apply(lambda x: literal_eval(x)[0])

In [67]:
def get_improvement(metric,dataframe):

    lookup = combined[combined.no_models==1]
    lookup['combination'] = lookup.combination.apply(lambda x: literal_eval(x)[0])
    lookup = lookup.groupby('combination')[metric].max().reset_index().set_index('combination')

    def get_max(lst,metric_):
        return max([lookup.loc[model][metric_].mean() for model in literal_eval(lst)])
    
    def get_min(lst,metric_):
        return min([lookup.loc[model][metric_].mean() for model in literal_eval(lst)])
    
    def get_mean(lst,metric_):
        l = [lookup.loc[model][metric_].mean() for model in literal_eval(lst)]
        return sum(l)/len(l)

    tdf = dataframe[dataframe.no_models!=1]
    tdf[f"max_{metric}"] = tdf.combination.apply(lambda x: get_max(x,metric))
    tdf[f"min_{metric}"] = tdf.combination.apply(lambda x: get_min(x,metric))
    tdf[f"mean_{metric}"] = tdf.combination.apply(lambda x: get_mean(x,metric))
    tdf['improvement'] = (tdf[metric] - tdf[f"max_{metric}"])*100 
    tdf['improvement'] = tdf.improvement.apply(lambda x: max(x,0))
    return tdf, lookup

tdf,_ = get_improvement('mcc',combined[~mask])


/tmp/ipykernel_74214/3253857108.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lookup['combination'] = lookup.combination.apply(lambda x: literal_eval(x)[0])
/tmp/ipykernel_74214/3253857108.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tdf[f"max_{metric}"] = tdf.combination.apply(lambda x: get_max(x,metric))
/tmp/ipykernel_74214/3253857108.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

Se

In [61]:
tdf[['alpha','mcc','max_mcc','min_mcc','mean_mcc', 'improvement']].corr()

,alpha,mcc,max_mcc,min_mcc,mean_mcc,improvement
alpha,1.000000,0.745988,0.194092,0.529285,0.636601,0.306519
mcc,0.745988,1.000000,0.233950,0.283499,0.406852,0.217622
max_mcc,0.194092,0.233950,1.000000,0.048977,0.434785,-0.025748
min_mcc,0.529285,0.283499,0.048977,1.000000,0.792674,0.226989
mean_mcc,0.636601,0.406852,0.434785,0.792674,1.000000,0.204552
improvement,0.306519,0.217622,-0.025748,0.226989,0.204552,1.000000


In [20]:
tdf.sort_values(by='mcc',ascending=False)[:20]

,Unnamed: 0,combination,alpha,mcc,f1,fbeta,p,r,no_models,type,max_mcc,min_mcc,mean_mcc,improvement
324,194,"['flan_ul2_label', 'llama_13b_label', 'mixtral...",0.484897,0.688538,0.850344,0.880104,0.901128,0.804979,5,2-shot-CoT,0.662384,0.536553,0.613476,2.615337
124,114,"['llama_13b_label', 'mixtral_8x7b_label', 'zep...",0.609427,0.686587,0.851553,0.876711,0.894325,0.812685,3,4-shot-CoT,0.662384,0.618682,0.636137,2.420253
123,113,"['llama_13b_label', 'mixtral_8x7b_label', 'ope...",0.531599,0.685568,0.851473,0.875749,0.892718,0.813871,3,2-shot-CoT,0.662384,0.618682,0.634494,2.318392
314,184,"['flan_ul2_label', 'llama_7b_label', 'llama_13...",0.471114,0.684697,0.848371,0.878422,0.899668,0.802608,5,2-shot-CoT,0.662384,0.536553,0.604117,2.231225
321,191,"['flan_ul2_label', 'llama_13b_label', 'mistral...",0.418771,0.684627,0.854523,0.870811,0.882019,0.828690,5,2-shot-CoT,0.662384,0.534017,0.594811,2.224242
315,185,"['flan_ul2_label', 'llama_7b_label', 'llama_13...",0.571916,0.683715,0.856191,0.867378,0.875000,0.838174,5,8-shot-CoT,0.662384,0.536553,0.605102,2.133088
324,194,"['flan_ul2_label', 'llama_13b_label', 'mixtral...",0.592391,0.682964,0.854179,0.869457,0.879950,0.829876,5,8-shot-CoT,0.662384,0.536553,0.613476,2.057923
324,194,"['flan_ul2_label', 'llama_13b_label', 'mixtral...",0.545545,0.682793,0.853472,0.870243,0.881795,0.826912,5,4-shot-CoT,0.662384,0.536553,0.613476,2.040845
124,114,"['llama_13b_label', 'mixtral_8x7b_label', 'zep...",0.564997,0.681020,0.849985,0.872824,0.888745,0.814464,3,2-shot-CoT,0.662384,0.618682,0.636137,1.863568
123,113,"['llama_13b_label', 'mixtral_8x7b_label', 'ope...",0.605543,0.680859,0.855156,0.865679,0.872840,0.838174,3,8-shot-CoT,0.662384,0.618682,0.634494,1.847459


In [85]:
import itertools

combs  = list(itertools.combinations(['llama_7b_label','openchat_label','zephyr_label','flan_ul2_label','llama_13b_label'],3))

In [86]:
combs

[('llama_7b_label', 'openchat_label', 'zephyr_label'),
 ('llama_7b_label', 'openchat_label', 'flan_ul2_label'),
 ('llama_7b_label', 'openchat_label', 'llama_13b_label'),
 ('llama_7b_label', 'zephyr_label', 'flan_ul2_label'),
 ('llama_7b_label', 'zephyr_label', 'llama_13b_label'),
 ('llama_7b_label', 'flan_ul2_label', 'llama_13b_label'),
 ('openchat_label', 'zephyr_label', 'flan_ul2_label'),
 ('openchat_label', 'zephyr_label', 'llama_13b_label'),
 ('openchat_label', 'flan_ul2_label', 'llama_13b_label'),
 ('zephyr_label', 'flan_ul2_label', 'llama_13b_label')]

In [87]:
combs = [set(list(comb)) for comb in combs]
dd=combined[combined.combination.apply(lambda x: set(literal_eval(x)) in combs)]

In [88]:
dd.soort_values

AttributeError: 'DataFrame' object has no attribute 'soort_values'

In [89]:
tdf,lookup = get_improvement('mcc',dd)
tdf['cot']=tdf['type'].apply(lambda x: 1 if "CoT" in x else 0)

/tmp/ipykernel_74214/3253857108.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lookup['combination'] = lookup.combination.apply(lambda x: literal_eval(x)[0])


In [74]:
lookup.sort_values(by='mcc')

,mcc
combination,
flan_t5_base_label,0.120410
flan_t5_large_label,0.335275
falcon_7b_label,0.344292
flan_t5_xl_label,0.415307
mistral_7b_label,0.534017
flan_ul2_label,0.536553
llama_7b_label,0.580548
llama_13b_label,0.618682
openchat_label,0.622417


In [92]:
tdf.sort_values(by='mcc',ascending=False)[:20]

,Unnamed: 0,combination,alpha,mcc,f1,fbeta,p,r,no_models,type,max_mcc,min_mcc,mean_mcc,improvement,cot
66,56,"['flan_ul2_label', 'llama_13b_label', 'opencha...",0.477110,0.661416,0.849747,0.851724,0.853047,0.846473,3,4-shot-CoT,0.622417,0.536553,0.592550,3.899946,1
125,115,"['llama_13b_label', 'openchat_label', 'zephyr_...",0.691063,0.661325,0.853566,0.844567,0.838673,0.868998,3,8-shot-CoT,0.627345,0.618682,0.622814,3.397975,1
67,57,"['flan_ul2_label', 'llama_13b_label', 'zephyr_...",0.510252,0.661175,0.844796,0.858592,0.868043,0.822762,3,4-shot-CoT,0.627345,0.536553,0.594193,3.382932,1
67,57,"['flan_ul2_label', 'llama_13b_label', 'zephyr_...",0.419557,0.657687,0.839963,0.860827,0.875321,0.807350,3,2-shot-CoT,0.627345,0.536553,0.594193,3.034189,1
66,56,"['flan_ul2_label', 'llama_13b_label', 'opencha...",0.387420,0.656881,0.838949,0.861150,0.876615,0.804386,3,2-shot-CoT,0.622417,0.536553,0.592550,3.446451,1
66,56,"['flan_ul2_label', 'llama_13b_label', 'opencha...",0.546279,0.656518,0.849485,0.846343,0.844262,0.854772,3,8-shot-CoT,0.622417,0.536553,0.592550,3.410152,1
112,102,"['llama_7b_label', 'llama_13b_label', 'opencha...",0.657021,0.654340,0.854226,0.832688,0.818923,0.892709,3,8-shot-CoT,0.622417,0.580548,0.607215,3.192332,1
113,103,"['llama_7b_label', 'llama_13b_label', 'zephyr_...",0.656120,0.653915,0.848127,0.845584,0.843897,0.852401,3,4-shot-CoT,0.627345,0.580548,0.608858,2.656965,1
67,57,"['flan_ul2_label', 'llama_13b_label', 'zephyr_...",0.564602,0.653340,0.845003,0.849886,0.853172,0.836989,3,8-shot-CoT,0.627345,0.536553,0.594193,2.599522,1
119,109,"['llama_7b_label', 'openchat_label', 'zephyr_l...",0.665818,0.651735,0.850621,0.837982,0.829763,0.872555,3,8-shot-CoT,0.627345,0.580548,0.610103,2.438952,1


In [76]:
tdf.sort_values(by='alpha', ascending=False)

,Unnamed: 0,combination,alpha,mcc,f1,fbeta,p,r,no_models,type,max_mcc,min_mcc,mean_mcc,improvement,cot
125,115,"['llama_13b_label', 'openchat_label', 'zephyr_...",0.691063,0.661325,0.853566,0.844567,0.838673,0.868998,3,8-shot-CoT,0.627345,0.618682,0.622814,3.397975,1
125,115,"['llama_13b_label', 'openchat_label', 'zephyr_...",0.667146,0.588782,0.811223,0.825574,0.835427,0.788382,3,8-shot,0.627345,0.618682,0.622814,0.000000,0
119,109,"['llama_7b_label', 'openchat_label', 'zephyr_l...",0.665818,0.651735,0.850621,0.837982,0.829763,0.872555,3,8-shot-CoT,0.627345,0.580548,0.610103,2.438952,1
113,103,"['llama_7b_label', 'llama_13b_label', 'zephyr_...",0.662351,0.641569,0.846774,0.832672,0.823529,0.871369,3,8-shot-CoT,0.627345,0.580548,0.608858,1.422342,1
119,109,"['llama_7b_label', 'openchat_label', 'zephyr_l...",0.660797,0.627331,0.840955,0.826269,0.816760,0.866627,3,4-shot-CoT,0.627345,0.580548,0.610103,0.000000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230,217,"['flan_ul2_label', 'llama_7b_label', 'mixtral_...",-0.094253,0.291034,0.748205,0.653060,0.602022,0.988145,3,zero-shot-system_prompt,0.662384,0.536553,0.593162,0.000000,0
286,273,"['llama_7b_label', 'mixtral_8x7b_label', 'open...",-0.101312,0.328142,0.729167,0.697723,0.678225,0.788382,3,zero-shot,0.662384,0.580548,0.621783,0.000000,0
234,221,"['flan_ul2_label', 'llama_13b_label', 'mixtral...",-0.111789,0.265079,0.743079,0.653626,0.605067,0.962656,3,zero-shot-system_prompt,0.662384,0.536553,0.605873,0.000000,0
281,268,"['llama_7b_label', 'llama_13b_label', 'opencha...",-0.173600,0.043338,0.710440,0.614075,0.563151,0.962063,3,zero-shot-system_prompt,0.622417,0.580548,0.607215,0.000000,0


In [77]:
import pandas as pd
import numpy as np
from scipy.stats import pearsonr

def calculate_significant_correlations(df, alpha=0.05):
    corr = df.corr()
    p_values = df.apply(lambda x: df.apply(lambda y: pearsonr(x, y)[1]))
    corr[p_values >= alpha] = np.nan
    return p_values,corr


p_values,significant_correlations = calculate_significant_correlations(tdf[['alpha', 'mcc','max_mcc','min_mcc','mean_mcc','improvement']])
round(significant_correlations,4)

,alpha,mcc,max_mcc,min_mcc,mean_mcc,improvement
alpha,1.0000,0.8361,NaN,NaN,NaN,0.3518
mcc,0.8361,1.0000,NaN,NaN,NaN,0.3934
max_mcc,NaN,NaN,1.0000,0.2780,0.6395,-0.2285
min_mcc,NaN,NaN,0.2780,1.0000,0.8717,NaN
mean_mcc,NaN,NaN,0.6395,0.8717,1.0000,NaN
improvement,0.3518,0.3934,-0.2285,NaN,NaN,1.0000
